## 0. 简介

本 Note 记录使用 Neural Networks (以下简称NN）做自然语言处理时所用的工具函数，包括：

1. `naive_preprocess(text)` 将输入文本字符串转换为单词 ID 的 Numpy 数组，并返回对应的单词-ID 字典和 ID-单词字典

2. `create_co_matrix(corpus, vocab_size, window_size=1)` 根据词表和窗口大小，在以单词 ID 表现的语料库上构建共现矩阵

3. `ppmi(C, verbose=False, eps= 1e-8)` 基于共现矩阵计算 PPMI (positive point-wise mutual information) 共现矩阵

4. `cos_similarity(x, y, eps=1e-8)` 计算两个向量之间的余弦相似度 

5. `most_similay(query, word_to_id, id_to_word, word_matrix, top=5)` 找出由词向量表示的 `word_matrix` 中和 `query` 最相近的 `top` 个单词

6. `convert_one_hot(corpus, vocab_size)` 将用单词 ID 表示的 `corpus` 转换为 one-hot 格式，其中 `corpus` 是一维或者二维的 Numpy 数组

## 1. naive_preprocess()

输入参数为文本序列，输出为该文本的单词 ID 序列，word_to_id 字典以及 id_to_word 字典

In [4]:
def naive_preprocess(text):
    '''基于输入文本构建语料库，附带构建单词-ID 和 ID-单词两份字典，仅处理以 . 结尾的句子文本
    
    :param text: 待处理的文本字符串
    
    :return corpus: 单词 ID 组成的 Numpy 数组
    :return word_to_id: key 是单词，value 是对应的 ID 的字典
    :return id_to_word: key 是 ID，value 是对应的单词的字典
    '''
    
    text = text.lower()
    text = text.replace('.', ' .') # 在文本末尾的句号前添加空格
    words = text.split(' ') # 以空格为定界符将输入文本拆分为单词
    
    word_to_id = {}
    id_to_word = {}
    
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id  # 核心就是构建 word_to-id 字典
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])  # 依 word_to_id 构建 corpus
        
    return corpus, word_to_id, id_to_word

## 2 create_co_matrix()

根据给定的词表和窗口，在语料库上统计构建贡献矩阵

主要参数包括：

1. `corpus`，提供统计数据的语料库，其中值是单词 ID
2. `vocab_size` 词汇表大小，共现矩阵是一个 (vocab_size, vocab_size) 的方阵
3. `window_size` 目标单词左右两边算作上下文的单词数目

返回的共现矩阵的行和列都是用 单词 ID 标定的，这进一步说明构建语料库时的 `word_to_id` 和 `id_to_word` 两个字典的重要性

In [1]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    '''基于 corpus 构建共现矩阵
    
    :param corpus: 单词 ID 形式的语料库
    :param vocab_size: 词汇表大小
    :param window_size: 上下文窗口大小，这里给出的是单边窗口大小，实际窗口要乘以 2
    
    :return: 基于次数的共现矩阵
    '''
    
    corpus_size = len(corpus_size)
    # 初始化共现矩阵
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    ## 关键步骤
    ## 遍历 corpus 中每个单词
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):  # 依次统计目标单词的上下文单词，按单边窗口计算单词下标范围
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1  # 注意这里是加法操作，因为是统计累计值
            
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix

## 3 ppmi()

基于次数的共现矩阵存在一个问题，某些诸如 **the** 等词语在文本库中会频繁出现，导致这些词会和其他词语多次共现，但这种共现但是没有太大的信息量。针对这个问题，引入了点间互信息（point-wise mutual information）的概念，如下：

$$
PMI(x,y) = \log_{2}\frac{P(x,y)}{P(x)P(y)}
$$

在共现矩阵上使用 PMI 时，可以将频率做为概率的近似来处理，若 $C(\cdot)$ 表示计数，$N$ 表示语料库中词语总数，那么如下：

$$
PMI(x,y) = \log_{2}\frac{C(x,y)\cdot N}{C(x)C(y)}
$$

需要注意的是，在具体实现时，通常将共现矩阵中所有项加起来作为语料库的词语总数 $N$，是实际数目的两倍，但是方便计算

另外，因为两个词语的共现次数可能为0，导致 PMI 值为 `-inf`，所以，一般采用 Positive PMI，即 PPMI，如下：

$$
PPMI(x, y) = max(0, PMI(x, y))
$$

In [7]:
def ppmi(C, verbose=False, eps=1e-8):
    '''构建 PPMI 矩阵
    
    Parameters
    ----------
    :param C: 共现矩阵
    :param Verbose: 是否显示转换进度
    :param eps: 避免出现 log0 情况
    
    :return: PPMI 矩阵
    '''
    
    M = np.zeros_like(C, dtype=np.float32)  # PPMI 矩阵
    N = np.sum(C)  # 计算语料库中的词汇量，实际是两倍的语料库词汇量
    S = np.sum(C, axis=0)  # 计算每一个单词在语料库中出现的次数
    
    total = C.shape[0] * C.shape[1]  # 总共需要执行的步骤数目
    cnt = 0  # 用于追踪进度
    
    ## 先行后列
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2((C[i, j] * N) / (S[i] * S[j]) + eps)
            M[i, j] = max(0, pmi)
            
            ## 处理输出进度，每 1% 输出依次
            ## 注意这里将 cnt 的计算放到条件判断中去，避免无意义计算
            if verbose:
                cnt +=1
                ## 每次完成 1%，输出
                if cnt % (total//100) == 0:
                    print('%.1f%% done', % (100*cnt/total))
    
    return M

SyntaxError: invalid syntax (<ipython-input-7-88426e7c912a>, line 32)

## 4. cos_similarity()

输入参数为两个向量，返回两个向量的余弦相似度

In [8]:
def cos_similarity(x, y, eps=1e-8):
    '''计算两个向量 x 和 y 的余弦相似度
    
    :param x: 向量 x
    :param y: 向量 y
    :param eps: 避免出现零向量的 L2 范数为 0 的情况
    
    :return: 返回 x 和 y 的余弦相似度
    '''
    
    ## x 和 y 执行正规化，除以各自的 L2 范数
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)  # eps 防止零向量导致的分母为 0 的情况
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)  # eps=1e-8 一般会在浮点数近似表示时被“吸收“”
    
    return np.dot(nx, ny)

## 5. most_similar()

给出 query 和 corpus，找出 corpus 中和 query 最相近的 top 个单词

实现中的关键是计算完词汇表所有与 `query` 的余弦值后，使用 `np.argsort()` 返回排序后的索引值

In [ ]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    '''在由词向量组成的 word_matrix 中搜索与单词 query 最相近的 top 个单词
    
    :param query: 目标词语
    :param word_to_id: 词语到 ID 的词典
    :param id_to_word: ID 到 词语的词典
    :param word_matirx: 所有词向量按行叠加的矩阵，单词用其语料库中 ID 索引
    :param top: 最相近的 top 个单词
    
    :return: 最相近的 top 个单词和对应的相似度
    '''
    
    if query not in word_to_id:
        print('%s is not found', % query)
        return
    
    print('[query] ' + query)
    
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    vocab_size = len(word_to_id)
    
    ## 存放和 query 的相似度的向量
    similarity = np.ones(vocab_size)
    
    ## 遍历 word_matrix，计算所有相似度
    for i in range(vocab_size):
        similarity[i] = cos_similarity(query_vec, word_matrix[i])
        
    ## 输出前 top 个单词
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue  # 跳过 query 自身
        print(' %s: %s', % (id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:
            return